In [4]:
import math
import pandas as pd

In [5]:
class DecisionTree:
    def __init__(self, df, target, positive, parent_val, parent):
        self.data = df
        self.target = target
        self.positive = positive
        self.parent_val = parent_val
        self.parent = parent
        self.childs = []
        self.decision = ''

    def _get_entropy(self, data):
        p = sum(data[self.target] == self.positive)
        n = data.shape[0] - p
        p_ratio = p / (p + n)
        n_ratio = 1 - p_ratio
        entropy_p = -p_ratio * math.log2(p_ratio) if p_ratio != 0 else 0
        entropy_n = -n_ratio * math.log2(n_ratio) if n_ratio != 0 else 0
        return entropy_p + entropy_n

    def _get_gain(self, feat):
        avg_info = sum(self._get_entropy(self.data[self.data[feat] == val]) *
                       sum(self.data[feat] == val) / self.data.shape[0]
                       for val in self.data[feat].unique())
        return self._get_entropy(self.data) - avg_info

    def _get_splitter(self):
        self.splitter = max(self.gains, key=lambda x: x[1])[0]

    def update_nodes(self):
        self.features = [col for col in self.data.columns if col != self.target]
        self.entropy = self._get_entropy(self.data)
        if self.entropy != 0:
            self.gains = [(feat, self._get_gain(feat)) for feat in self.features]
            self._get_splitter()
            residual_columns = [k for k in self.data.columns if k != self.splitter]
            for val in self.data[self.splitter].unique():
                df_tmp = self.data[self.data[self.splitter] == val][residual_columns]
                tmp_node = DecisionTree(df_tmp, self.target, self.positive, val, self.splitter)
                tmp_node.update_nodes()
                self.childs.append(tmp_node)

In [6]:

def print_tree(node, depth=0):
    if node:
        print(f"{'  ' * depth}Parent: {node.parent} | Parent Value: {node.parent_val}")
        for child in node.childs:
            print_tree(child, depth + 1)


In [7]:
df = pd.read_excel('exa.xlsx')
dt = DecisionTree(df, 'Play', 'Yes', '', '')
dt.update_nodes()
print_tree(dt)

Parent:  | Parent Value: 
  Parent: Outlook | Parent Value: Sunny
    Parent: Humidity | Parent Value: High
    Parent: Humidity | Parent Value: Normal
  Parent: Outlook | Parent Value: Overcast
  Parent: Outlook | Parent Value: Rainy
    Parent: WindSpeed | Parent Value: Weak
    Parent: WindSpeed | Parent Value: Strong
